In [1]:

import numpy as np
import matplotlib.pyplot as plt
#matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

#%%
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools

#%%
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image
#%%

def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 98).resize([128,128])).flatten() / 255.0
#%%
X_train = [] # ELA converted images
X_test = [] # ELA converted images
Y_train = [] # 0 for fake, 1 for real 
Y_test = [] # 0 for fake, 1 for real 

In [2]:
!pwd

/gpfsdswork/projects/rech/xkq/udw91ac/CODES


In [5]:
#%%
import random
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Au_Train/' 
j=0
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        
        if filename.endswith('jpg') or filename.endswith('bmp'):
            j=j+1;
            full_path = os.path.join(dirname, filename)
            X_train.append(prepare_image(full_path))
            Y_train.append(0)
            if len(Y_train) % 500 == 0:
                print(f'Processing {len(Y_train)} images')
                print(j)
                
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Tp_Train/' 
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        j=j+1;
        if filename.endswith('jpg') or filename.endswith('tif'):
            full_path = os.path.join(dirname, filename)
            X_train.append(prepare_image(full_path))
            Y_train.append(1)
            if len(Y_train) % 500 == 0:
                print(f'Processing {len(Y_train)} images')
                print(j)

#random.shuffle(X)
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Au_Test/' 
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        j=j+1;
        if filename.endswith('jpg') or filename.endswith('bmp'):
            full_path = os.path.join(dirname, filename)
            X_test.append(prepare_image(full_path))
            Y_test.append(0)
            if len(Y_test) % 500 == 0:
                print(f'Processing {len(Y_test)} images')
                print(j)
                
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Tp_Test/' 
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        j=j+1;
        if filename.endswith('jpg') or filename.endswith('tif'):
            full_path = os.path.join(dirname, filename)
            X_test.append(prepare_image(full_path))
            Y_test.append(1)
            if len(Y_test) % 500 == 0:
                print(f'Processing {len(Y_test)} images')
                print(j)

                
#%%

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = to_categorical(Y_train, 2)

Y_test = to_categorical(Y_test, 2)
X_train = X_train.reshape(-1, 128, 128, 3)
X_test = X_test.reshape(-1, 128, 128, 3)
#%%

#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
#X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_test), len(Y_test))

#%%

Processing 500 images
500
Processing 1000 images
1000
Processing 1500 images
1500
Processing 2000 images
2000


/gpfslocalsup/pub/anaconda-py3/2020.02/envs/tensorflow-gpu-2.3.0/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)
/gpfslocalsup/pub/anaconda-py3/2020.02/envs/tensorflow-gpu-2.3.0/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag)


Processing 2500 images
2500
Processing 3000 images
3000
Processing 3500 images
3500
Processing 4000 images
4000
Processing 4500 images
4500
Processing 5000 images
5000
Processing 5500 images
5500
Processing 6000 images
6000
Processing 6500 images
6500
Processing 7000 images
7000
Processing 7500 images
7500
Processing 8000 images
8000
Processing 8500 images
8500
Processing 9000 images
9000
Processing 9500 images
9500
Processing 10000 images
10000
Processing 500 images
10591
Processing 1000 images
11091
Processing 1500 images
11591
Processing 2000 images
12091
Processing 2500 images
12591
10091 10091
2523 2523


In [6]:

def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
   
    model.add(Conv2D(filters = 32, kernel_size = (7, 7), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))

    model.add(MaxPool2D(pool_size = (2, 2)))
   
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model


from keras.optimizers import Adam


from keras.callbacks import ModelCheckpoint, EarlyStopping
#%%
model = build_model()
model.summary()
epochs = 100
batch_size = 64
init_lr = 1e-5
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
#%%


checkpoint = ModelCheckpoint("model_best_98.h5", monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
early_stopping = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 300, verbose = 0, mode = 'auto')
#%%
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                validation_data = (X_test, Y_test),
                callbacks = [checkpoint,early_stopping])

#%%
model.save('model_casia_best.h5')
#%%

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 118, 118, 32)      50208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 59, 59, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 111392)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               2

In [3]:
from tensorflow import keras

from keras.models import load_model
saved_model = keras.models.load_model("/gpfsdswork/projects/rech/xkq/udw91ac/CODES/model_best_98.h5")

In [4]:
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Tp_Test/'
B=[]
j=0
m=0
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        j=j+1;
        if filename.endswith('jpg') or filename.endswith('tif'):
            full_path = os.path.join(dirname, filename)
            X=(prepare_image(full_path))
            X = np.array(X)
            X= X.reshape(-1, 128, 128, 3)
            A=saved_model(X)
            if A[0][1] < A[0][0]:
                B=filename
                m=m+1

print(m)

23


In [5]:
path = '/gpfsdswork/projects/rech/xkq/udw91ac/DATABASE/Au_Test/' 
B=[]
j=0
k=0
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        j=j+1;
        if filename.endswith('jpg') or filename.endswith('bmp'):
            full_path = os.path.join(dirname, filename)
            X=(prepare_image(full_path))
            X = np.array(X)
            X= X.reshape(-1, 128, 128, 3)
            A=saved_model(X)
            if A[0][1] > A[0][0]:
                B=filename
                k=k+1         

print(k)

173


In [5]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

NameError: name 'keras' is not defined

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=3,generator=traindata, validation_data= testdata, validation_steps=10,epochs=3,callbacks=[checkpoint,early])

In [ ]:

def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
   
    model.add(Conv2D(filters = 32, kernel_size = (7, 7), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))

    model.add(MaxPool2D(pool_size = (2, 2)))
   
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

#%%
model = build_model()
model.summary()
epochs = 100
batch_size = 64
init_lr = 1e-5
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
#%%
early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0, patience = 20, verbose = 0, mode = 'auto')
#%%
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                validation_data = (X_test, Y_test),
                callbacks = [early_stopping])

#%%
model.save('model_casia_run2.h5')
#%%